In [24]:
from datasets import load_dataset, DatasetDict

dataset_name = "dbpedia_14"
dataset = load_dataset(dataset_name, split="test")
dataset

Found cached dataset dbpedia_14 (/admin/home-augustas/.cache/huggingface/datasets/dbpedia_14/dbpedia_14/2.0.0/01dab9e10d969eadcdbc918be5a09c9190a24caeae33b10eee8f367a1e3f1f0c)


Dataset({
    features: ['label', 'title', 'content'],
    num_rows: 70000
})

In [28]:
# Add a unique index to each row
def add_index(example, idx):
    example['idx'] = idx
    return example

processed_dataset = dataset.map(add_index, with_indices=True)

print(processed_dataset[0])
print(len(set(processed_dataset["idx"])))

processed_dataset = DatasetDict({ "test": processed_dataset })
print(processed_dataset)

print(processed_dataset["test"]["idx"][:10])

print(dataset.features.keys())

# Save the dataset
# processed_dataset.push_to_hub(f"AugustasM/{dataset_name}_with_indices", private=True)

Loading cached processed dataset at /admin/home-augustas/.cache/huggingface/datasets/dbpedia_14/dbpedia_14/2.0.0/01dab9e10d969eadcdbc918be5a09c9190a24caeae33b10eee8f367a1e3f1f0c/cache-fe7abc7bcca974aa.arrow
Pushing split test to the Hub.


{'label': 0, 'title': 'TY KU', 'content': " TY KU /taɪkuː/ is an American alcoholic beverage company that specializes in sake and other spirits. The privately-held company was founded in 2004 and is headquartered in New York City New York. While based in New York TY KU's beverages are made in Japan through a joint venture with two sake breweries. Since 2011 TY KU's growth has extended its products into all 50 states.", 'idx': 0}
70000
DatasetDict({
    test: Dataset({
        features: ['label', 'title', 'content', 'idx'],
        num_rows: 70000
    })
})
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
dict_keys(['label', 'title', 'content'])


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/70 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
from datasets import load_dataset
from collections import Counter

# dataset = load_dataset("ai2_arc", "ARC-Challenge", split="test")
dataset = load_dataset("truthfulqa_mc", split="validation")
print(dataset)

# Majority class baseline
counts = Counter(dataset["label"])
print(counts)
most_freq_label = counts.most_common(1)[0][0]
print(most_freq_label)
print(f"Majority class baseline accuracy: {counts[most_freq_label] / len(dataset) * 100:.2f}")

Found cached dataset hellaswag (/admin/home-augustas/.cache/huggingface/datasets/hellaswag/default/0.1.0/512a66dd8b1b1643ab4a48aa4f150d04c91680da6a4096498a5e5f799623d5ae)


Dataset({
    features: ['ind', 'activity_label', 'ctx_a', 'ctx_b', 'ctx', 'endings', 'source_id', 'split', 'split_type', 'label'],
    num_rows: 10042
})
Counter({'2': 2584, '0': 2515, '1': 2485, '3': 2458})
2
Majority class baseline accuracy: 25.73


In [12]:
from datasets import load_dataset
from collections import Counter

dataset = load_dataset("ai2_arc", "ARC-Challenge", split="test")
print(dataset)

print(dataset[0]["choices"]["label"])
print(dataset[0]["answerKey"])

dataset = dataset.map(lambda example: { "label": example["choices"]["label"].index(example["answerKey"]) })
print(dataset["label"][:10])

# Majority class baseline
counts = Counter(dataset["label"])
print(counts)
most_freq_label = counts.most_common(1)[0][0]
print(most_freq_label)
print(f"Majority class baseline accuracy: {counts[most_freq_label] / len(dataset) * 100:.2f}")

Found cached dataset ai2_arc (/admin/home-augustas/.cache/huggingface/datasets/ai2_arc/ARC-Challenge/1.0.0/1569c2591ea2683779581d9fb467203d9aa95543bb9b75dcfde5da92529fd7f6)
Loading cached processed dataset at /admin/home-augustas/.cache/huggingface/datasets/ai2_arc/ARC-Challenge/1.0.0/1569c2591ea2683779581d9fb467203d9aa95543bb9b75dcfde5da92529fd7f6/cache-ed441d7b93e96de2.arrow


Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 1172
})
['A', 'B', 'C', 'D']
C
[2, 1, 2, 3, 3, 1, 2, 2, 1, 0]
Counter({1: 311, 2: 310, 3: 285, 0: 266})
1
Majority class baseline accuracy: 26.54


In [14]:
print(dataset["label"][:10])

# Majority class baseline
counts = Counter(dataset["label"])
print(counts)
most_freq_label = counts.most_common(1)[0][0]
print(most_freq_label)
print(f"Majority class baseline accuracy: {counts[most_freq_label] / len(dataset) * 100:.2f}")

[2, 1, 2, 3, 3, 1, 2, 2, 1, 0]
Counter({1: 311, 2: 310, 3: 285, 0: 266})
1
Majority class baseline accuracy: 26.54


In [6]:
from datasets import concatenate_datasets, DatasetDict

dataset = load_dataset("amazon_polarity", split="test")

# Get a stratified split
processed_dataset = dataset.shuffle(seed=42)

# Take the first n samples
pos_samples = processed_dataset.filter(lambda x: x["label"] == 1, num_proc=12).select(range(50000))
neg_samples = processed_dataset.filter(lambda x: x["label"] == 0, num_proc=12).select(range(50000))

# Join train and test splits into one
processed_dataset = concatenate_datasets([pos_samples, neg_samples]).shuffle(seed=42)

# Majority class baseline
counts = Counter(processed_dataset["label"])
print(counts)
most_freq_label = counts.most_common(1)[0][0]
print(most_freq_label)
print(f"Majority class baseline accuracy: {counts[most_freq_label] / len(processed_dataset) * 100:.2f}")

processed_dataset = DatasetDict({ "test": processed_dataset })
print(processed_dataset)

print(processed_dataset["test"]["label"][:10])

# Save the dataset
# processed_dataset.push_to_hub(f"AugustasM/amazon_polarity_subsample", private=True)

Found cached dataset amazon_polarity (/admin/home-augustas/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc)
Loading cached shuffled indices for dataset at /admin/home-augustas/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc/cache-0a5d0b47b5e8dfc6.arrow
Loading cached processed dataset at /admin/home-augustas/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc/cache-f82de97f5c546404_*_of_00012.arrow
Loading cached processed dataset at /admin/home-augustas/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc/cache-3755e47a04d615ff_*_of_00012.arrow
Loading cached shuffled indices for dataset at /admin/home-augustas/.cache/huggingface/datasets/amazon_polarity/amazon_polari

Counter({1: 50000, 0: 50000})
1
Majority class baseline accuracy: 50.00
DatasetDict({
    test: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 100000
    })
})


Pushing split test to the Hub.


[1, 1, 1, 0, 0, 1, 1, 0, 0, 0]


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
dataset = load_dataset("AugustasM/amazon_polarity_subsample")
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /admin/home-augustas/.cache/huggingface/datasets/AugustasM___parquet/AugustasM--amazon_polarity_subsample-be3d73de71c20033/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 100000
    })
})

## Test model sizes

In [1]:
from transformers import AutoModel, AutoConfig

In [2]:
model_name = "microsoft/deberta-v3-large"
model_name = "microsoft/deberta-v2-xxlarge"
model_name = "EleutherAI/gpt-j-6b"
model_name = "databricks/dolly-v2-3b"
model_name = "gpt2"
model_name = "gpt2-xl"
# model_name = "allenai/unifiedqa-v2-t5-11b-1363200"
# model_name = "allenai/unifiedqa-v2-t5-3b-1363200"
# model_name = "allenai/unifiedqa-v2-t5-large-1363200"
# model_name = "allenai/unifiedqa-v2-t5-base-1363200"
# model_name = "allenai/unifiedqa-v2-t5-small-1363200"

model_cfg = AutoConfig.from_pretrained(model_name)
model_cfg.is_encoder_decoder, model_cfg.is_decoder

(False, False)

In [3]:
model_cfg.torch_dtype

In [4]:
model_cfg.architectures

['GPT2LMHeadModel']

In [5]:
model_cfg.max_position_embeddings

1024

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer

GPT2TokenizerFast(name_or_path='gpt2-xl', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)

In [7]:
model = AutoModel.from_pretrained(model_name)
print(f"# params: {sum(p.numel() for p in model.parameters()):,d}")

# params: 1,557,611,200


In [11]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained(model_name)
print(f"# params: {sum(p.numel() for p in model.parameters()):,d}")

# params: 737,668,096


In [11]:
from transformers import GPTNeoXForCausalLM

model = GPTNeoXForCausalLM.from_pretrained(model_name)
print(f"# params: {sum(p.numel() for p in model.parameters()):,d}")

# params: 2,775,086,080


In [45]:
model.gpt_neox.embed_in

Embedding(50280, 2560)

In [42]:
import torch

model.to("cuda:0")

# More than 2048 does not work
x = torch.randint(0, tokenizer.vocab_size, (2, 2048)).to("cuda:0")
x

with torch.no_grad():
    outputs = model(x)

outputs.keys()

odict_keys(['logits', 'past_key_values'])

In [43]:
outputs.logits.shape

torch.Size([2, 2048, 50280])